In [4]:
import os, glob, csv, warnings
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from shapely.ops import unary_union
from fiona.env import Env
from pykrige.ok import OrdinaryKriging
from numpy.linalg import LinAlgError
from scipy.spatial import cKDTree

# -------- Paths
DATA_DIR   = r"D:\Work\SpatialCARE\Daily\DailyGPKG"
PASIG_SHP  = r"C:\Users\HP\Desktop\SpatialCARE\Pasig\Pasig.shp"
ROADS_SHP  = r"C:\Users\HP\Desktop\SpatialCARE\Pasig\PasigRN.shp" 
OUT_F_DIR  = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours"
OUT_CSV    = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\exceedance_local_summary.csv"
os.makedirs(OUT_F_DIR, exist_ok=True)

# -------- Settings
WGS84, UTM51 = "+proj=longlat +datum=WGS84 +no_defs", "+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs"
FIG_SIZE, FIG_DPI = (7,7), 150
GRID_RES_M = 100
VARIOGRAM_MODEL, RETRY_JITTER = "spherical", 1e-3
COVERAGE_MAX_DIST_M = 3000

# Local guideline bins
BINS    = [0.0, 25.0, 35.0, 45.0, 55.0, 90.0, np.inf]
LABELS  = ["Good (0–25)","Fair (25.1–35)","Unhealthy (sens.) (35.1–45)",
           "Very unhealthy (45.1–55)","Acutely unhealthy (55.1–90)","Emergency (≥91)"]
COLORS  = ["#00E400","#FFFF00","#FF7E00","#FF0000","#8F3F97","#7E0023"]
CMAP_CAT = mpl.colors.ListedColormap(COLORS)
NORM_CAT = mpl.colors.BoundaryNorm(BINS, ncolors=len(COLORS), clip=False)

# Contours to label
CONTOURS = [25, 35, 55, 90]

def pick_pm_col(cols):
    for c in cols:
        if str(c).lower() in ("pm25","pm_25","pm2_5","pm2.5"): return c
    return None

# -------- Boundary
with Env(SHAPE_RESTORE_SHX="YES"):
    pasig = gpd.read_file(PASIG_SHP)
if pasig.crs is None: pasig = pasig.set_crs(WGS84)
pasig_utm = pasig.to_crs(UTM51)
geom = unary_union(pasig_utm.geometry)

# Grid
minx,miny,maxx,maxy = geom.bounds
xs = np.arange(minx, maxx+GRID_RES_M, GRID_RES_M)
ys = np.arange(miny, maxy+GRID_RES_M, GRID_RES_M)
grid_x, grid_y = np.meshgrid(xs, ys)

# -------- Roads
with Env(SHAPE_RESTORE_SHX="YES"):
    roads = gpd.read_file(ROADS_SHP)
if roads.crs is None:
    roads = roads.set_crs(WGS84)
roads_utm = roads.to_crs(UTM51)

road_col = None
for c in ("highway","fclass","type"):
    if c in roads_utm.columns:
        road_col = c; break
if road_col is None:
    raise SystemExit("No road class column found in roads shapefile.")

roads_utm[road_col] = roads_utm[road_col].astype(str).str.lower()
wanted = {"primary","secondary","tertiary"}
roads_sel = roads_utm[roads_utm[road_col].isin(wanted)].copy()
try:
    roads_clip = gpd.clip(roads_sel, pasig_utm)
except Exception:
    roads_clip = gpd.overlay(roads_sel, pasig_utm[["geometry"]], how="intersection")

STYLE_MAP = {
    "primary":   {"linewidth": 2.0, "color": "#d62728"},
    "secondary": {"linewidth": 1.6, "color": "#1f77b4"},
    "tertiary":  {"linewidth": 1.2, "color": "#2ca02c"},
}
ROAD_HANDLES = [
    Line2D([0],[0], color=STYLE_MAP["primary"]["color"],   lw=STYLE_MAP["primary"]["linewidth"],   label="Primary"),
    Line2D([0],[0], color=STYLE_MAP["secondary"]["color"], lw=STYLE_MAP["secondary"]["linewidth"], label="Secondary"),
    Line2D([0],[0], color=STYLE_MAP["tertiary"]["color"],  lw=STYLE_MAP["tertiary"]["linewidth"],  label="Tertiary"),
]

# -------- CSV header
if not os.path.exists(OUT_CSV):
    with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([
            "date","stations",
            "pct_gt25","pct_gt35","pct_gt55","pct_ge91",
            "pct_good","pct_fair","pct_unsens","pct_very","pct_acute","pct_emerg",
            "total_area_km2"
        ])

# -------- Iterate daily files
files = sorted(glob.glob(os.path.join(DATA_DIR, "date_2025-*.gpkg")))
if not files: raise SystemExit("No daily GPKG files found.")

for f in files:
    base = os.path.splitext(os.path.basename(f))[0]
    day  = base.replace("date_","")

    g = gpd.read_file(f)
    if g.crs is None: g = g.set_crs(WGS84)
    if ("geometry" not in g) or g.geometry.is_empty.all():
        lon = next((c for c in g.columns if str(c).lower() in ("longitude","lon","x")), None)
        lat = next((c for c in g.columns if str(c).lower() in ("latitude","lat","y")), None)
        if lon and lat:
            g = g.set_geometry(gpd.points_from_xy(g[lon], g[lat], crs=g.crs))
        else:
            print("Skip (no geometry):", base); continue

    pm_col = pick_pm_col(g.columns)
    if pm_col is None: print("Skip (no PM):", base); continue

    pts = g.to_crs(UTM51)[[pm_col,"geometry"]].copy()
    pts["x"]=pts.geometry.x; pts["y"]=pts.geometry.y
    pts = pts.groupby(["x","y"], as_index=False)[pm_col].mean()
    x = pts["x"].values; y = pts["y"].values; z = np.clip(pts[pm_col].astype(float).values, 0, None)
    if len(z)<3: print("Skip (n<3):", base); continue

    # Kriging
    try:
        OK = OrdinaryKriging(x,y,z, variogram_model=VARIOGRAM_MODEL,
                             enable_plotting=False, verbose=False, coordinates_type="euclidean")
        zg, zv = OK.execute("grid", xs, ys, backend="loop"); zg = np.array(zg)
    except LinAlgError:
        z_jitter = z + np.random.normal(0, RETRY_JITTER, size=z.shape)
        OK = OrdinaryKriging(x,y,z_jitter, variogram_model=VARIOGRAM_MODEL,
                             enable_plotting=False, verbose=False, coordinates_type="euclidean")
        zg, zv = OK.execute("grid", xs, ys, backend="loop"); zg = np.array(zg)

    # Masks
    tree = cKDTree(np.c_[x,y]); dists,_ = tree.query(np.c_[grid_x.ravel(), grid_y.ravel()], k=1)
    cover = dists.reshape(grid_x.shape) <= COVERAGE_MAX_DIST_M
    centers = gpd.GeoSeries(gpd.points_from_xy(grid_x.ravel(), grid_y.ravel()), crs=UTM51)
    inside  = centers.within(geom).values.reshape(zg.shape)
    zmask = np.where(cover & inside, np.maximum(zg,0.0), np.nan)

    if not np.isfinite(zmask).any():
        print("Skip (no valid cells):", base); continue

    # Percent area by thresholds
    cell_area_km2 = (GRID_RES_M*GRID_RES_M)/1e6
    total_area_km2 = np.isfinite(zmask).sum()*cell_area_km2
    pct_gt25 = 100.0*(np.nan_to_num(zmask>25).sum()*cell_area_km2)/max(1e-9,total_area_km2)
    pct_gt35 = 100.0*(np.nan_to_num(zmask>35).sum()*cell_area_km2)/max(1e-9,total_area_km2)
    pct_gt55 = 100.0*(np.nan_to_num(zmask>55).sum()*cell_area_km2)/max(1e-9,total_area_km2)
    pct_ge91 = 100.0*(np.nan_to_num(zmask>=91).sum()*cell_area_km2)/max(1e-9,total_area_km2)

    # Percent by categories
    idx = np.digitize(zmask, BINS, right=True) - 1
    cats = [(idx==i).sum() for i in range(6)]
    pct_by_cat = [100.0*(c*cell_area_km2)/max(1e-9,total_area_km2) for c in cats]

    # Plot categorical with contours + roads
    cat_mask = np.ma.masked_where(~np.isfinite(zmask), idx)
    fig, ax = plt.subplots(figsize=FIG_SIZE, dpi=FIG_DPI)
    ax.imshow(cat_mask, extent=[xs[0], xs[-1], ys[0], ys[-1]],
              origin="lower", cmap=CMAP_CAT, norm=NORM_CAT)
    pasig_utm.boundary.plot(ax=ax, color="black", linewidth=0.8)
    for klass, style in STYLE_MAP.items():
        sub = roads_clip.loc[roads_clip[road_col]==klass]
        if not sub.empty: sub.plot(ax=ax, **style, zorder=3)

    try:
        cs = ax.contour(grid_x, grid_y, zmask, levels=CONTOURS,
                        colors="black", linewidths=0.9, linestyles=["--","-","-","-"])
        ax.clabel(cs, fmt=lambda v: f"{int(v)} µg/m³", inline=True, fontsize=8)
    except Exception:
        pass

    subtitle = f"%>25: {pct_gt25:.1f} • %>35: {pct_gt35:.1f} • %>55: {pct_gt55:.1f} • %≥91: {pct_ge91:.1f} • Stations: {len(z)}"
    ax.set_title(f"PM₂.₅ (Local guideline) • {day}\n{subtitle}")
    ax.set_axis_off()

    # Legend: categories + roads
    cat_handles = [mpatches.Patch(color=c, label=l) for c,l in zip(COLORS, LABELS)]
    all_handles = cat_handles + ROAD_HANDLES
    fig.legend(handles=all_handles, loc="upper center",
               bbox_to_anchor=(0.5, 0.20), ncol=3,
               frameon=True, fontsize=8, title="Legend")
    plt.subplots_adjust(bottom=0.30)

    out = os.path.join(OUT_F_DIR, f"{base}_kriged_localContours.png")
    plt.savefig(out, bbox_inches="tight"); plt.close(fig)
    print("Saved:", out)

    # CSV row
    with open(OUT_CSV, "a", newline="", encoding="utf-8") as fcsv:
        csv.writer(fcsv).writerow([
            day, len(z),
            pct_gt25, pct_gt35, pct_gt55, pct_ge91,
            *pct_by_cat, total_area_km2
        ])


Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-06_kriged_localContours.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-07_kriged_localContours.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-08_kriged_localContours.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-09_kriged_localContours.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-10_kriged_localContours.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-11_kriged_localContours.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-12_kriged_localContours.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\date_2025-02-13_kriged_localContours.png
Saved: C:\Users\HP\Desktop\SpatialCARE\Outputs\f

In [5]:
import os
import glob
from PIL import Image

# Folder where your PNG files are located
img_dir = r"C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours"

# Get all .png files sorted by filename (important for order)
png_files = sorted(glob.glob(os.path.join(img_dir, "*.png")))

# Load images
frames = [Image.open(f) for f in png_files]

# Output file
gif_path = os.path.join(img_dir, "kriging_local_contours.gif")

# Save as GIF
frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=500,   # duration per frame in milliseconds (adjust speed here)
    loop=0          # 0 means infinite loop
)

print(f"GIF saved at: {gif_path}")


GIF saved at: C:\Users\HP\Desktop\SpatialCARE\Outputs\figures\kriging_local_contours\kriging_local_contours.gif
